In [44]:
import pandas as pd

In [45]:
from google.colab import files


In [46]:
# --- 1) Importer ton fichier CSV depuis ton ordinateur ---
uploaded = files.upload()

Saving gares-de-voyageurs (1).csv to gares-de-voyageurs (1) (1).csv


In [47]:
# Récupérer automatiquement le nom du fichier téléchargé
file_name = list(uploaded.keys())[0]

In [48]:
# --- 2) Charger la base ---
df = pd.read_csv(file_name, sep=";", encoding="utf-8")  # auto-détection du séparateur

df.head()

,Nom,Trigramme,Segment(s) DRG,Position géographique,Code commune,Code(s) UIC
0,Abancourt,ABT,C,"49.6852237, 1.7743058",60001,87313759
1,Abbaretz,AAR,C,"47.5546432, -1.5244159",44001,87481614
2,Abbeville,ABB,B,"50.10221, 1.82449",80001,87317362
3,Ablon-sur-Seine,ABL,B,"48.725468, 2.419151",94001,87545269
4,Achères Grand Cormier,GCR,B,"48.9551835, 2.0919031",78551,87386052


In [49]:
df.head()
df.columns


Index(['Nom', 'Trigramme', 'Segment(s) DRG', 'Position géographique',
       'Code commune', 'Code(s) UIC'],
      dtype='object')

In [50]:
df.columns = df.columns.str.strip().str.replace("\ufeff", "")  # enlève BOM + espaces
df.columns


Index(['Nom', 'Trigramme', 'Segment(s) DRG', 'Position géographique',
       'Code commune', 'Code(s) UIC'],
      dtype='object')

In [51]:
df.head()


,Nom,Trigramme,Segment(s) DRG,Position géographique,Code commune,Code(s) UIC
0,Abancourt,ABT,C,"49.6852237, 1.7743058",60001,87313759
1,Abbaretz,AAR,C,"47.5546432, -1.5244159",44001,87481614
2,Abbeville,ABB,B,"50.10221, 1.82449",80001,87317362
3,Ablon-sur-Seine,ABL,B,"48.725468, 2.419151",94001,87545269
4,Achères Grand Cormier,GCR,B,"48.9551835, 2.0919031",78551,87386052


In [52]:
# --- 2) Liste des villes + département correspondant ---
villes_departements = {
    # Bouches-du-Rhône (13)
    "Arles": "Bouches-du-Rhône",
    "Martigues": "Bouches-du-Rhône",
    "La Ciotat": "Bouches-du-Rhône",
    "Aubagne": "Bouches-du-Rhône",
    "Cassis": "Bouches-du-Rhône",
    "Carry-le-Rouet": "Bouches-du-Rhône",
    "Sausset-les-Pins": "Bouches-du-Rhône",
    "Ensuès": "Bouches-du-Rhône",

    # Var (83)
    "Sanary-sur-Mer": "Var",
    "Bandol": "Var",
    "Saint-Raphaël": "Var",
    "Fréjus": "Var",
    "Hyères": "Var",
    "Toulon": "Var",
    "Les Arcs": "Var",
    "Vidauban": "Var",
    "Le Luc": "Var",

    # Vaucluse (84)
    "Avignon": "Vaucluse",
    "Orange": "Vaucluse",
    "L’Isle-sur-la-Sorgue": "Vaucluse",
    "Cavaillon": "Vaucluse",
    "Carpentras": "Vaucluse",

    # Alpes-de-Haute-Provence (04)
    "Manosque": "Alpes-de-Haute-Provence",
    "Oraison": "Alpes-de-Haute-Provence",
    "La Brillanne": "Alpes-de-Haute-Provence",
    "Volx": "Alpes-de-Haute-Provence",
    "Sisteron": "Alpes-de-Haute-Provence",

    # Hautes-Alpes (05)
    "Gap": "Hautes-Alpes",
    "Briançon": "Hautes-Alpes",
    "Embrun": "Hautes-Alpes",
    "Veynes": "Hautes-Alpes",
    "Montdauphin": "Hautes-Alpes",

    # Alpes-Maritimes (06)
    "Cannes": "Alpes-Maritimes",
    "Antibes": "Alpes-Maritimes",
    "Menton": "Alpes-Maritimes",
    "Grasse": "Alpes-Maritimes",
    "Villefranche-sur-Mer": "Alpes-Maritimes",
    "Tende": "Alpes-Maritimes",
    "Saorge": "Alpes-Maritimes",
    "Breil-sur-Roya": "Alpes-Maritimes"
}

# --- 3) Filtrer les lignes dont la colonne 'Nom' contient l’une de ces villes ---
pattern = "|".join(villes_departements.keys())
df_filtre = df[df["Nom"].str.contains(pattern, case=False, na=False)]

# --- 4) Ajouter la colonne "Département" ---
def get_departement(nom_gare):
    for ville, dep in villes_departements.items():
        if ville.lower() in nom_gare.lower():
            return dep
    return None

df_filtre["Département"] = df_filtre["Nom"].apply(get_departement)

# --- 5) Afficher le résultat ---
df_filtre.head()

# --- 6) Enregistrer la nouvelle base filtrée ---
df_filtre.to_csv("gares_filtrees_paca.csv", index=False)
files.download("gares_filtrees_paca.csv")

/tmp/ipython-input-3360115074.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtre["Département"] = df_filtre["Nom"].apply(get_departement)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
!pip install folium


In [54]:
import pandas as pd
from google.colab import files

print("⬆️ Upload le CSV des gares PACA")
uploaded_gares = files.upload()
gares_file = list(uploaded_gares.keys())[0]

print("⬆️ Upload le CSV des horaires d'ouverture")
uploaded_horaires = files.upload()
horaires_file = list(uploaded_horaires.keys())[0]

df_gares = pd.read_csv(gares_file, encoding="utf-8")
df_horaires = pd.read_csv(horaires_file, encoding="utf-8")


⬆️ Upload le CSV des gares PACA


Saving gares_filtrees_paca (4).csv to gares_filtrees_paca (4).csv
⬆️ Upload le CSV des horaires d'ouverture


Saving horaires-des-gares1 (1).csv to horaires-des-gares1 (1) (2).csv


In [55]:
df_horaires.columns

Index(['﻿UIC;Gare;Jour de la semaine;Horaire en jour normal;Horaire en jour férié'], dtype='object')

In [58]:
df_horaires = pd.read_csv(
    "horaires-des-gares1 (1).csv",
    sep=';',
    encoding='utf-8-sig'
)

print(df_horaires.columns)

Index(['UIC', 'Gare', 'Jour de la semaine', 'Horaire en jour normal',
       'Horaire en jour férié'],
      dtype='object')


In [59]:
df_gares.columns

Index(['Nom', 'Trigramme', 'Segment(s) DRG', 'Position géographique',
       'Code commune', 'Code(s) UIC', 'Département'],
      dtype='object')

In [61]:
# Renommer pour plus de clarté
df_gares = df_gares.rename(columns={"Code(s) UIC": "UIC"})

# Séparer les UIC multiples (ex: "87981902;87318964")
df_gares["UIC"] = df_gares["UIC"].astype(str).str.split(";")

# Dupliquer les lignes (une ligne = un UIC)
df_gares = df_gares.explode("UIC")

# Nettoyer et convertir en int
df_gares["UIC"] = df_gares["UIC"].astype(int)

print(df_gares[["Nom", "UIC"]].head())


                                Nom       UIC
0      Aéroport Charles de Gaulle 1  87271460
1  Aéroport Charles de Gaulle 2 TGV  87001479
1  Aéroport Charles de Gaulle 2 TGV  87271494
2                           Antibes  87757674
3                             Arles  87753657


In [62]:
df_horaires["UIC"] = df_horaires["UIC"].astype(int)


In [63]:
df_horaires_filtre = df_horaires[
    df_horaires["UIC"].isin(df_gares["UIC"])
]


In [64]:
df_final = df_horaires_filtre.merge(
    df_gares,
    on="UIC",
    how="left"
)


In [65]:
df_final.columns


Index(['UIC', 'Gare', 'Jour de la semaine', 'Horaire en jour normal',
       'Horaire en jour férié', 'Nom', 'Trigramme', 'Segment(s) DRG',
       'Position géographique', 'Code commune', 'Département'],
      dtype='object')

In [66]:
df_final = df_final[[
    "Nom",
    "Département",
    "Code commune",
    "Position géographique",
    "UIC",
    "Jour de la semaine",
    "Horaire en jour normal",
    "Horaire en jour férié"
]]


In [67]:
ordre_jours = [
    "Lundi", "Mardi", "Mercredi",
    "Jeudi", "Vendredi", "Samedi", "Dimanche"
]

df_final["Jour de la semaine"] = pd.Categorical(
    df_final["Jour de la semaine"],
    categories=ordre_jours,
    ordered=True
)

df_final = df_final.sort_values(
    by=["Nom", "Jour de la semaine"]
)


In [68]:
df_final.to_csv("gares_PACA_horaires_geo.csv", index=False)
df_final.head(10)


,Nom,Département,Code commune,Position géographique,UIC,Jour de la semaine,Horaire en jour normal,Horaire en jour férié
168,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Lundi,05:10-23:30,05:10-00:30
81,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Mardi,05:10-23:30,05:10-00:30
169,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Mercredi,05:10-23:30,05:10-00:30
75,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Jeudi,05:10-23:30,05:10-00:30
200,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Vendredi,05:10-23:30,05:10-00:30
16,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Samedi,05:40-00:30,05:10-00:30
91,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Dimanche,05:10-00:30,05:10-00:30
216,Arles,Bouches-du-Rhône,13004,"43.684853, 4.63203",87753657,Lundi,05:45-22:30,06:45-22:30
156,Arles,Bouches-du-Rhône,13004,"43.684853, 4.63203",87753657,Mardi,05:45-22:30,06:45-22:30
57,Arles,Bouches-du-Rhône,13004,"43.684853, 4.63203",87753657,Mercredi,05:45-22:30,06:45-22:30


In [69]:
df_final = df_final[df_final["UIC"] != 87271494]


In [70]:
df_final.to_csv("gares_PACA_horaires_geo_clean.csv", index=False)


In [71]:
# Séparer latitude et longitude
df_final[["Latitude", "Longitude"]] = df_final["Position géographique"].str.split(
    ",", expand=True
)

# Convertir en float
df_final["Latitude"] = df_final["Latitude"].astype(float)
df_final["Longitude"] = df_final["Longitude"].astype(float)

df_final.head()


,Nom,Département,Code commune,Position géographique,UIC,Jour de la semaine,Horaire en jour normal,Horaire en jour férié,Latitude,Longitude
168,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Lundi,05:10-23:30,05:10-00:30,43.585784,7.119498
81,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Mardi,05:10-23:30,05:10-00:30,43.585784,7.119498
169,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Mercredi,05:10-23:30,05:10-00:30,43.585784,7.119498
75,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Jeudi,05:10-23:30,05:10-00:30,43.585784,7.119498
200,Antibes,Alpes-Maritimes,6004,"43.585784, 7.119498",87757674,Vendredi,05:10-23:30,05:10-00:30,43.585784,7.119498


In [72]:
import folium

# Centre approximatif PACA
map_paca = folium.Map(
    location=[43.7, 6.0],
    zoom_start=7,
    tiles="OpenStreetMap"
)


In [73]:
df_gares_map = df_final.drop_duplicates(subset=["UIC"])


In [74]:
for _, row in df_gares_map.iterrows():
    popup_text = f"""
    <b>{row['Nom']}</b><br>
    Département : {row['Département']}<br>
    UIC : {row['UIC']}
    """

    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=popup_text,
        icon=folium.Icon(icon="train", prefix="fa")
    ).add_to(map_paca)


In [52]:
map_paca


In [75]:
gares = df_final.copy()

# garder une ligne par gare
gares = gares.drop_duplicates(subset=["UIC"])

gares[["Nom", "Latitude", "Longitude"]].head()


,Nom,Latitude,Longitude
168,Antibes,43.585784,7.119498
216,Arles,43.684853,4.632030
69,Aubagne,43.296045,5.566580
6,Avignon Centre,43.941900,4.805281
87,Avignon TGV,43.921586,4.786079


In [76]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))


In [77]:
import json

def troncon_proche(troncon_geojson, gares, seuil_km=2):
    coords = json.loads(troncon_geojson)["coordinates"]
    for lon, lat in coords:
        for _, gare in gares.iterrows():
            if haversine(lat, lon, gare["Latitude"], gare["Longitude"]) <= seuil_km:
                return True
    return False


In [79]:
print("⬆️ Upload le CSV des gares PACA")
uploaded_lines = files.upload()
lines_file = list(uploaded_gares.keys())[0]

troncons = pd.read_csv(lines_file, encoding="utf-8")

⬆️ Upload le CSV des gares PACA


Saving lignes-par-region-administrative (1).csv to lignes-par-region-administrative (1).csv


In [84]:
import pandas as pd

troncons = pd.read_csv("lignes-par-region-administrative (1).csv", sep=';', engine='python')

# Nettoyer les noms de colonnes (espaces et caractères invisibles)
troncons.columns = troncons.columns.str.strip().str.replace('\xa0', ' ')
print(troncons.columns)


Index(['CODE_LIGNE', 'LIB_LIGNE', 'REGION', 'RG_TRONCON', 'PKD', 'PKF',
       'IDGAIA', 'X_D_L93', 'Y_D_L93', 'X_F_L93', 'Y_F_L93', 'X_D_WGS84',
       'Y_D_WGS84', 'X_F_WGS84', 'Y_F_WGS84', 'C_GEO_D', 'C_GEO_F',
       'Geo Point', 'Geo Shape'],
      dtype='object')


In [87]:
!pip install haversine

In [88]:
import json
from haversine import haversine

def troncon_proche(troncon_geojson, gares, seuil_km=2):
    geo = json.loads(troncon_geojson)
    coords = geo["coordinates"]

    # si MultiLineString, "coords" est une liste de listes de listes
    if geo["type"] == "MultiLineString":
        liste_points = [pt for segment in coords for pt in segment]  # aplatir
    else:  # LineString
        liste_points = coords

    for lon, lat in liste_points:
        for _, gare in gares.iterrows():
            if haversine((lat, lon), (gare["Latitude"], gare["Longitude"])) <= seuil_km:
                return True
    return False


In [90]:
troncons_filtrés = troncons[
    troncons["Geo Shape"].apply(lambda g: troncon_proche(g, gares))
]


In [97]:
import folium
import json
import matplotlib.cm as cm
import matplotlib.colors as colors

# Créer la carte centrée sur les gares
centre_lat = gares["Latitude"].mean()
centre_lon = gares["Longitude"].mean()
m = folium.Map(location=[centre_lat, centre_lon], zoom_start=10)

# Ajouter les gares
for _, gare in gares.iterrows():
    folium.Marker(
        location=[gare["Latitude"], gare["Longitude"]],
        popup=gare["Nom"],
        icon=folium.Icon(color='blue', icon='train', prefix='fa')
    ).add_to(m)

# Palette de couleurs foncées
n_lignes = troncons_filtrés["LIB_LIGNE"].nunique()
cmap = cm.get_cmap("Dark2", n_lignes)  # palette foncée
ligne_to_color = {ligne: colors.rgb2hex(cmap(i)[:3]) for i, ligne in enumerate(troncons_filtrés["LIB_LIGNE"].unique())}

# Ajouter les tronçons avec la couleur correspondante
for _, troncon in troncons_filtrés.iterrows():
    geojson = json.loads(troncon["Geo Shape"])
    couleur = ligne_to_color[troncon["LIB_LIGNE"]]
    folium.GeoJson(
        geojson,
        style_function=lambda x, col=couleur: {'color': col, 'weight': 3},
        tooltip=f"Ligne: {troncon['LIB_LIGNE']}"
    ).add_to(m)

# Sauvegarder la carte
m.save("carte_gares_lignes_couleurs_foncees.html")


/tmp/ipython-input-615404988.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("Dark2", n_lignes)  # palette foncée


In [98]:
m